In [9]:
import pandas as pd
import numpy as np
from keras.layers import LSTM, Activation, Dropout, Dense, Input
from keras.layers.embeddings import Embedding
from keras.models import Model
import string
import re
from keras.preprocessing.text import Tokenizer
from sklearn.preprocessing import LabelBinarizer
from keras.preprocessing.sequence import pad_sequences
import keras
from sklearn.model_selection import train_test_split

In [10]:
df=pd.read_csv('https://raw.githubusercontent.com/lazy-af-7/twitter_airline_nlp/main/Train.csv')
df.head()

,Unnamed: 0,tweet_id,airline_sentiment,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,0,569612379811676161,negative,US Airways,NaN,09202010,NaN,0,@USAirways US Airlines 699 LA to RDU is holdin...,NaN,2015-02-22 13:39:08 -0800,NaN,Mountain Time (US & Canada)
1,1,570283248301043712,negative,Southwest,NaN,slobotski,NaN,0,@SouthwestAir went to purchase a flight that I...,NaN,2015-02-24 10:04:56 -0800,Midwest + Airplanes,Central Time (US & Canada)
2,2,569684565838553088,negative,American,NaN,61jr,NaN,0,@AmericanAir their flights into Buffalo as wel...,NaN,2015-02-22 18:25:59 -0800,St. Catharines,NaN
3,3,569313126342123520,negative,US Airways,NaN,iZoom23,NaN,0,@USAirways Sitting in a cesspool of germs on t...,NaN,2015-02-21 17:50:01 -0800,New York,Eastern Time (US & Canada)
4,4,569723892358467584,negative,United,NaN,ljtypes,NaN,0,@united you advertise the flight and its still...,NaN,2015-02-22 21:02:15 -0800,H-town,NaN


In [11]:
df=df[['airline_sentiment','text']]
df.head()

,airline_sentiment,text
0,negative,@USAirways US Airlines 699 LA to RDU is holdin...
1,negative,@SouthwestAir went to purchase a flight that I...
2,negative,@AmericanAir their flights into Buffalo as wel...
3,negative,@USAirways Sitting in a cesspool of germs on t...
4,negative,@united you advertise the flight and its still...


In [14]:
stopwords = [ "a", "about", "above", "after", "again", "against", "all", "am", "an", "and", "any", "are", "as", "at", "be", "because", 
             "been", "before", "being", "below", "between", "both", "but", "by", "could", "did", "do", "does", "doing", "down", "during",
             "each", "few", "for", "from", "further", "had", "has", "have", "having", "he", "he'd", "he'll", "he's", "her", "here", 
             "here's", "hers", "herself", "him", "himself", "his", "how", "how's", "i", "i'd", "i'll", "i'm", "i've", "if", "in", "into",
             "is", "it", "it's", "its", "itself", "let's", "me", "more", "most", "my", "myself", "nor", "of", "on", "once", "only", "or",
             "other", "ought", "our", "ours", "ourselves", "out", "over", "own", "same", "she", "she'd", "she'll", "she's", "should", 
             "so", "some", "such", "than", "that", "that's", "the", "their", "theirs", "them", "themselves", "then", "there", "there's",
             "these", "they", "they'd", "they'll", "they're", "they've", "this", "those", "through", "to", "too", "under", "until", "up",
             "very", "was", "we", "we'd", "we'll", "we're", "we've", "were", "what", "what's", "when", "when's", "where", "where's",
             "which", "while", "who", "who's", "whom", "why", "why's", "with", "would", "you", "you'd", "you'll", "you're", "you've",
             "your", "yours", "yourself", "yourselves" ]
data=df.copy()
def remove_stopwords(data):
  data['review without stopwords'] = data['text'].apply(lambda x : ' '.join([word for word in x.split() if word not in (stopwords)]))
  return data

def remove_tags(string):
    result = re.sub('<.*?>','',string)
    return result
    
data_without_stopwords = remove_stopwords(data)
data_without_stopwords['clean_review']= data_without_stopwords['review without stopwords'].apply(lambda cw : remove_tags(cw))
data_without_stopwords['clean_review'] = data_without_stopwords['clean_review'].str.replace('[{}]'.format(string.punctuation), ' ')

In [23]:
tweet_list = []
for i in range(len(data['text'])):
  tweet_list.append(data_without_stopwords['clean_review'])
 
sentiment = data_without_stopwords['airline_sentiment']

In [25]:
print(tweet_list[0])

0        USAirways US Airlines 699 LA RDU holding onto...
1        SouthwestAir went purchase flight I began pro...
2        AmericanAir flights Buffalo well    You fligh...
3        USAirways Sitting cesspool germs ground  PHL ...
4        united advertise flight still website still c...
                              ...                        
7681     USAirways I promise  I can help I will nvr fl...
7682    “ USAirways  Reminder  From 2 28  we’ll tweeti...
7683     USAirways flight 1898 landed 2 hours ago stil...
7684          united alright thank you  Much appreciated 
7685     united btw   Virgin  JetBlue managed time dep...
Name: clean_review, Length: 7686, dtype: object


In [34]:
y = np.array(list(map(lambda x: 1 if x=="positive" else 0, sentiment)))

X_train, X_test,Y_train, Y_test = train_test_split(tweet_list[0], y, test_size=0.2, random_state = 45)

In [35]:
print(X_train)

7157     SouthwestAir thank much completely made thing...
5793                           united still waiting reply
5853     AmericanAir flt 7a tom  I now rec d notificat...
3208     AmericanAir  united wrote back saying  it s c...
2974     AmericanAir seats assigned inappropriate chil...
                              ...                        
3616     USAirways flight 4524 delayed 2 hours deadhea...
6012     united flight 4841   3 gate changes top this ...
5763                  VirginAmerica save 871 tomorrow AM 
6558     USAirways   I m Cust Svc line  Your Charlotte...
7115                   AmericanAir boarding 2mins YAY    
Name: clean_review, Length: 6148, dtype: object


In [36]:
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(X_train)

words_to_index = tokenizer.word_index

In [37]:
def read_glove_vector(glove_vec):
  with open(glove_vec, 'r', encoding='UTF-8') as f:
    words = set()
    word_to_vec_map = {}
    for line in f:
      w_line = line.split()
      curr_word = w_line[0]
      word_to_vec_map[curr_word] = np.array(w_line[1:], dtype=np.float64)



  return word_to_vec_map

In [38]:
word_to_vec_map = read_glove_vector('/content/drive/MyDrive/Twitter NLP/glove.twitter.27B.100d.txt')

maxLen = 150

In [39]:
vocab_len = len(words_to_index)
embed_vector_len = word_to_vec_map['moon'].shape[0]

emb_matrix = np.zeros((vocab_len, embed_vector_len))

for word, index in words_to_index.items():
  embedding_vector = word_to_vec_map.get(word)
  if embedding_vector is not None:
    emb_matrix[index, :] = embedding_vector

embedding_layer = Embedding(input_dim=vocab_len, output_dim=embed_vector_len, input_length=maxLen, weights = [emb_matrix], trainable=False)

In [40]:
def imdb_rating(input_shape):

  X_indices = Input(input_shape)

  embeddings = embedding_layer(X_indices)

  X = LSTM(128, return_sequences=True)(embeddings)

  X = Dropout(0.6)(X)

  X = LSTM(128, return_sequences=True)(X)

  X = Dropout(0.6)(X)

  X = LSTM(128)(X)

  X = Dense(1, activation='sigmoid')(X)

  model = Model(inputs=X_indices, outputs=X)

  return model

In [47]:

model = imdb_rating((maxLen,))
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 150)]             0         
_________________________________________________________________
embedding (Embedding)        (None, 150, 100)          932900    
_________________________________________________________________
lstm_3 (LSTM)                (None, 150, 128)          117248    
_________________________________________________________________
dropout_2 (Dropout)          (None, 150, 128)          0         
_________________________________________________________________
lstm_4 (LSTM)                (None, 150, 128)          131584    
_________________________________________________________________
dropout_3 (Dropout)          (None, 150, 128)          0         
_________________________________________________________________
lstm_5 (LSTM)                (None, 128)               1315

In [49]:

X_train_indices = tokenizer.texts_to_sequences(X_train)

X_train_indices = pad_sequences(X_train_indices, maxlen=maxLen, padding='post')

In [51]:
adam = keras.optimizers.Adam(learning_rate = 0.0001)
model.compile(optimizer=adam, loss='binary_crossentropy', metrics=['accuracy'])

model.fit(X_train_indices, Y_train, batch_size=64, epochs=8)

Epoch 1/8
97/97 [==============================] - 97s 945ms/step - loss: 0.4481 - accuracy: 0.8372
Epoch 2/8
97/97 [==============================] - 97s 1s/step - loss: 0.4488 - accuracy: 0.8358
Epoch 3/8
97/97 [==============================] - 93s 963ms/step - loss: 0.4471 - accuracy: 0.8366
Epoch 4/8
97/97 [==============================] - 94s 964ms/step - loss: 0.4479 - accuracy: 0.8355
Epoch 5/8
97/97 [==============================] - 93s 963ms/step - loss: 0.4444 - accuracy: 0.8386
Epoch 6/8
97/97 [==============================] - 94s 968ms/step - loss: 0.4342 - accuracy: 0.8447
Epoch 7/8
97/97 [==============================] - 94s 965ms/step - loss: 0.4515 - accuracy: 0.8332
Epoch 8/8
97/97 [==============================] - 93s 962ms/step - loss: 0.4431 - accuracy: 0.8390


In [55]:
test=pd.read_csv('https://raw.githubusercontent.com/lazy-af-7/twitter_airline_nlp/main/Train.csv')
test=test[['text']]
X_test=test['text'].tolist()
X_test_indices = tokenizer.texts_to_sequences(X_test)

X_test_indices = pad_sequences(X_test_indices, maxlen=maxLen, padding='post')
preds = model.predict(X_test_indices)

predicted=[]
for i in range(len(preds)):
  if preds[i] > 0.6:
    predicted.append('positive')
  elif preds[i] < 0.4:
    predicted.append('negative')
  else :
    predicted.append('neutral')

In [56]:
final=pd.DataFrame()
final["Id"]=test.index
final['Predicted']=predicted
final.to_csv('out.csv',index=False)